In [13]:
import pandas as pd
import numpy as np
import os
import transformers
import random
import math
import json
import ipynbname
import re
import glob
from itertools import chain
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5ForConditionalGeneration
from datetime import date, datetime

import spacy, statistics
from spacy.tokens import Doc
from nltk.corpus import sentiwordnet as swn
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings('ignore')

nlp = spacy.load("en_core_web_sm")
sia = SentimentIntensityAnalyzer()

In [3]:
# GLOBAL VARIABLES
test_name = '100Agents100Timesteps_Jan26'
connectionCap = 0.3 #Percentage the of total number of agents
influencerThreshold = 0.97 #Percentage of connection cap an agent must have for it to be considered an influencer

simDuration = 100
numAgents = 100
numInfluencer = 1
numAgents_half = math.ceil(numAgents/2) + numInfluencer

In [4]:
# AGENT SEEDS
load_agents = True #Turn to False if agents will be loaded from social media data
seed_path = '/Users/zacpangan/Documents/Research/Data/Simulation Data'
#preferenceDataFrame_positive = pd.read_excel('~/Documents/Research/Data/Simulation Data/mask_tweets_seed_positive.xlsx')
#preferenceDataFrame_negative = pd.read_excel('~/Documents/Research/Data/Simulation Data/mask_tweets_seed_negative.xlsx')
preferenceDataFrame_positive = pd.read_csv('%s/mask_tweets_seed_positive.csv' % (seed_path))
preferenceDataFrame_negative = pd.read_csv('%s/mask_tweets_seed_negative.csv' % (seed_path))


#Subject of Interest sought for in the simulation
subjectInterest = 'mask'
subjectSynonyms = ['masks', 'masking']
subjectNeutralKnowledge = 'Wearing a mask is part of the measures deployed by the government to aid in stopping the spread of a virus.'

#An influencer just repeats the initial message all throughout the simulation
influencerMessage = "Wearing masks does not help in stopping the virus, it's useless and I hate it." 

preferenceDataFrame = preferenceDataFrame_positive
#preferenceDataFrame = pd.concat([preferenceDataFrame_positive.iloc[:numAgents_half],preferenceDataFrame_negative.iloc[:numAgents_half]], axis=0, ignore_index=True)
preferenceDataFrame.rename(columns={'Text':'knowledge'}, inplace=True)

In [11]:
# DIRECTORY MANAGEMENT
from datetime import datetime
root = '/Users/zacpangan/Documents/Research/Code/Models/AgentNetworkSimulation'
script_name = str(ipynbname.name())

file_path = os.path.join(root, script_name)
date_time = str(datetime.now().strftime("%m-%d-%Y %H_%M_%S"))
date_path = os.path.join(root, script_name, date_time)
test_path = os.path.join(root, script_name, date_time, test_name)
ind_results = "Individual Agent Convos"   
ind_results_path = os.path.join(root, script_name, date_time, test_name, ind_results)

try: 
    os.mkdir(file_path) 
except OSError as error: 
    print(error) 

try:   
    os.mkdir(date_path) 
    os.mkdir(test_path)
    os.mkdir(ind_results_path)
except OSError as error:
    print(error) 

load_script = "1 InfluencerSimulation - Jan2023-jvsc-9bd93d30-1f56-4cf1-acc3-ce44668ba50394cd1338-d3ce-49e4-885e-8aa63800e031"
load_time = str("01-26-2023 22_27_22")#("%m-%d-%Y %H_%M_%S"))
load_test = "100Agents10Timesteps_TestRun2_Jan26"

load_path = os.path.join(root, load_script, load_time, load_test, ind_results)

[Errno 17] File exists: '/Users/zacpangan/Documents/Research/Code/Models/AgentNetworkSimulation/2 LoadAgentsSimulation - Jan2023-jvsc-4c4f69b3-1e70-4813-9483-eaa3946bd8c644e6d802-9bb8-4a69-a267-4b604b9a48aa'


In [7]:
#i=0
for row in preferenceDataFrame_negative['Text']:
   rowSentiment = sia.polarity_scores(row)['compound']
   print(rowSentiment)
   #i = i + 1
   #if(rowSentiment> -0.1):
      #preferenceDataFrame_negative = preferenceDataFrame_negative.drop(i)
   #else:
      #print(rowSentiment)
preferenceDataFrame_negative.to_csv('%s/mask_tweets_seed_negative.csv' % (seed_path), "w", index=False)

0.0
0.0
-0.7964
0.0
0.8316
-0.7203
-0.2323
0.3612
-0.4939
-0.5507
-0.8343
-0.765
0.0258


In [8]:
class Agent:
    def __init__(self, name, preference, connections, chats, hist, influenceStat, conversationRec):
        self.name = int(name) #Agent Number
        self.preference = preference #An actual document stored in a DF correspoding to the Agent Number, will serve as first message.
        self.connections = connections #A list of  other agents this agent is following
        self.chats = chats #A dictionary of all the chat lists this person has received from all the other agents its connected
        self.history = hist #A list of chat history this agent with another agent it is connected to
        self.influenceStatus = influenceStat
        self.conversationRecord = conversationRec
        #self.sentimentRecord = []

    def sendChat(self, target, message):
        if target.name not in self.chats.keys():
            self.chats[target.name] =  [message]
        else:
            self.chats[target.name].append(message)
        
        self.conversationRecord = self.conversationRecord.append({'sender': self.name, 'receiver': target.name, 'message': message}, ignore_index=True)  
        self.history.append(message)

    def receiveChat(self, source, message):
        if source.name not in self.chats.keys():
            self.chats[source.name] =  [message]
        else:
            self.chats[source.name].append(message)
        
        self.conversationRecord = self.conversationRecord.append({'sender': source.name, 'receiver': self.name, 'message': message}, ignore_index=True)
        self.history.append(message)

In [86]:
class SocialNetwork:
    def __init__(self):
        self.agentDict = {}
        self.agentMatrix = pd.DataFrame(columns=['agent', 'agent_preference', 'num_connections', 'agent_connections', 'influence_status'])
        self.correspondenceTimeline = {}
        self.model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/GODEL-v1_1-base-seq2seq")
        
    def referenceConnections(self, agent, connections):
        return agent in connections

    def generateAgents(self, numAgents): #Generates a number of random agents depending on the value inputted   
        #Define the number of connections an agent can have in the current environment
        maxConnections = int(math.floor(connectionCap*numAgents))
    
        for i in range(len(self.agentDict)+1,len(self.agentDict)+numAgents+1):
            #Prepare random connections per agent
            numConnections = random.randint(1,maxConnections)
            possibleConnections = list(range(1,numAgents+1))
            possibleConnections.remove(i)
            agentConnections = random.sample(possibleConnections, numConnections)
            influenceStatus = False
            agentPreference = preferenceDataFrame['knowledge'][i]
            #Prepare preference per agent
            
            #Instantiate Agent in a list with the index as the name, preference, connections, empty chathistory list, and an empty chat dictonary.
            convoRec = pd.DataFrame(columns=['sender','receiver','message'])
            self.agentDict[i] = Agent(str(i), agentPreference, agentConnections, {}, [], influenceStatus,convoRec)
            
            #Capture Agent instatiation data
            self.agentMatrix.loc[i]= {'agent': i,'agent_preference': agentPreference, 'num_connections': numConnections, 'agent_connections': agentConnections, 'influence_status': influenceStatus}
            
            #Confirm Agent creation    
            #print('Agent %d has been created with %d outgoing connections.' % (i, numConnections))
            
            filteredDictionary = dict(filter(lambda x:x[0] in self.agentDict[i].connections, self.agentDict.items()))
            self.agentDict[i].connections = [v for v in filteredDictionary.values()]
            
    def loadAgents(self):
        agent_files = filter(os.path.isfile, glob.glob(load_path + '.csv'))
        agent_files = sorted(agent_files, key = os.path.getmtime)
        #agent_files.sort()
        print(agent_files)
        i=1
        
        # loop over the list of csv files
        for agent in agent_files:
            # read the csv file
            with open(os.path.join(load_path, agent), 'r') as f:
                agentConvo = pd.DataFrame(pd.read_csv(f))
            
            agentPreference = agentConvo['message'][1]
            agentConnections = list(agentConvo['receiver'].unique())
            
            if (i in agentConnections): agentConnections.remove(i)
            agentChats = {}
            agentHistory = list(agentConvo['message'])
            agentInfluenceStatus = False
            self.agentDict[i] = Agent(str(i), agentPreference, agentConnections, {}, [], agentInfluenceStatus, agentConvo)
            
            filteredDictionary = dict(filter(lambda x:x[0] in self.agentDict[i].connections, self.agentDict.items()))
            self.agentDict[i].connections = [v for v in filteredDictionary.values()]
            
            i = i+1
    
    def generateInfluencers(self, numInfluencers):
        
        numRandomAgents = len(self.agentDict)
        influencerStartID = numRandomAgents+1
        maxConnections = int(math.floor(connectionCap*numAgents))
        for i in range(numInfluencers):
            numConnections = maxConnections
            possibleConnections = list(range(1,numAgents+i))
            agentConnections = random.sample(possibleConnections, numConnections)
            influenceStatus = True
            agentPreference = influencerMessage
            
            #Instantiate Agent in a list with the index as the name, preference, connections, empty chathistory list, and an empty chat dictonary.
            self.agentDict[influencerStartID+i] = Agent(str(influencerStartID+i), agentPreference, agentConnections, {}, [], influenceStatus)
            
            #Capture Agent instatiation data
            self.agentMatrix.loc[influencerStartID+i]= {'agent': influencerStartID+i,'agent_preference': agentPreference, 'num_connections': numConnections, 'agent_connections': agentConnections, 'influence_status': influenceStatus}
            
            #Confirm Agent creation    
            #print('Agent %d has been created with %d outgoing connections.' % (influencerStartID+i, numConnections)) 
            
            filteredDictionary = dict(filter(lambda x:x[0] in self.agentDict[influencerStartID+i].connections, self.agentDict.items()))
            self.agentDict[influencerStartID+i].connections = [v for v in filteredDictionary.values()]     
            
    def sendMessage(self, source, target, message):
        if source.influenceStatus == True:
            message = influencerMessage #An influencer just repeats the initial message all throughout the simulation
        if target in source.connections:
            self.agentDict[source.name].sendChat(self.agentDict[target.name],message)
            self.agentDict[target.name].receiveChat(self.agentDict[source.name],message)
            print('Agent %s has sent a message to Agent %s.' % (source.name, target.name))
        #Enter try-catch here, but this should not even happen as we control the simulation 
        return ({'source':source.name, 'target': target.name, 'message': message})
        
    def firstBatchMessage(self):
        list(map(lambda x,y,z: [self.sendMessage(x,i,z) for i in y], 
                        {k: v for k, v in self.agentDict.items()}.values(),
                        {k: v.connections for k, v in self.agentDict.items()}.values(), 
                        {k: v.preference for k, v in self.agentDict.items()}.values()))
       #print(flogs)
    
    def createMessage(self, source, target):
        # Instruction for a chitchat task
        instruction = f'Instruction: given a dialog context and related knowledge, you need to response safely based on the knowledge.'
        if source.history != '':
            knowledge = '[KNOWLEDGE] ' + subjectNeutralKnowledge + ' '.join(map(str, source.history[-20:]))
        dialog = source.chats[target.name] #' EOS '.join(str(chain.from_iterable(source.chats[target.name])))
        query = f"{instruction} [CONTEXT] {dialog} {knowledge}"
        input_ids = self.tokenizer(f"{query}", return_tensors="pt").input_ids
        outputs = self.model.generate(input_ids, max_length=128, min_length=8, top_p=0.9, do_sample=True)
        output = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return output
    
    def agentCreateSendMessage(self, agent):
        correspondenceLogs= list(map(lambda x,y: self.sendMessage(self.agentDict[agent.name],x,y),
                        self.agentDict[agent.name].connections,
                        list(map(lambda q: self.createMessage(self.agentDict[agent.name],q),
                        self.agentDict[agent.name].connections))))
        return(correspondenceLogs)
     
    def batchCreateSendMessage(self):
        batchCorrespondenceLogs = list(map(lambda x: self.agentCreateSendMessage(x), self.agentDict.values()))
        self.correspondenceTimeline[len(self.correspondenceTimeline)+1] = list(chain.from_iterable(batchCorrespondenceLogs))
        #print(logs)
    
    def subjectInterestCheck(self, doc):
        #Replace subjectSynonyms with subjectInterest 
        exp = r'({})'.format('|'.join(subjectSynonyms))
        doc =[re.sub(exp, subjectInterest, chat) for chat in doc]    
        #Check if the nominal subject is the subjectInterest 
        
        check = lambda doc: True if subjectInterest in [
            token.lower_ for token in doc 
            if token.dep_ == 'nsubj' 
            and token.is_alpha
        ] else False
        return check
    
    def sentimentCheck(self, agent):
        #use the subjectInterestCheck function before applying the rule-based sentiment analysis
        relevant_chats = list(filter(lambda x: self.subjectInterestCheck(x), agent.history[-20:]))
        if(relevant_chats == []):
            sentimentScore = 0
        else:
            sentimentScore = sia.polarity_scores(relevant_chats)['compound']
             # Use the Sentiment Intensity Analyser from the compound sentiment of Vader sentiment to get a sentimentScore per timestep
        #sentimentTuple = [agent.name, sentimentScore]
       
        return sentimentScore
    
    def preferenceSentimentCheck(self, agent):
        sentimentScore = sia.polarity_scores(agent.preference)['compound']
        return sentimentScore
    
    def sentimentRecord(self, timestep):
        
        if (timestep < 1):
            sentiLogs = list(map(lambda x: [self.preferenceSentimentCheck(x)],
                {k: v for k, v in self.agentDict.items()}.values()))
        else:
            sentiLogs = list(map(lambda x: [self.sentimentCheck(x)],
                {k: v for k, v in self.agentDict.items()}.values()))
        
        return (list(chain.from_iterable(sentiLogs)))   
       

# Main Method While Loop

In [ ]:
timestep = 0
twitter = SocialNetwork()

sentimentDF = pd.DataFrame(columns=range(numAgents+numInfluencer))
twitter.generateAgents(numAgents)
twitter.generateInfluencers(numInfluencer)
sentimentDF = sentimentDF.append([twitter.sentimentRecord(timestep)], ignore_index = True)
print('==================== Timestep: '+ str(timestep) +' ====================')

timestep = timestep + 1
twitter.firstBatchMessage()
sentimentDF = sentimentDF.append([twitter.sentimentRecord(timestep)], ignore_index = True)
print('==================== Timestep: '+ str(timestep) +' ====================')

while timestep <= simDuration:
    twitter.batchCreateSendMessage()
    sentimentDF = sentimentDF.append([twitter.sentimentRecord(timestep)], ignore_index = True)
    timestep = timestep + 1
    print('==================== Timestep: '+ str(timestep) +' ====================')
    

In [87]:
timestep = 0
twitter = SocialNetwork()
#sentimentDF = pd.DataFrame(columns=range(numAgents+numInfluencer))

In [88]:
twitter.loadAgents()

[]


In [101]:
agent_files = lambda x: os.path.isfile(os.path.join(load_path, x)), os.listdir(load_path) 
print(agent_files)
agent_files = sorted(agent_files, key = lambda x: os.path.getmtime(os.path.join(load_path, x)))


(<function <lambda> at 0x3079e6700>, ['Agent45_Convos.csv', 'Agent72_Convos.csv', 'Agent27_Convos.csv', 'Agent10_Convos.csv', 'Agent91_Convos.csv', 'Agent59_Convos.csv', 'Agent8_Convos.csv', 'Agent48_Convos.csv', 'Agent80_Convos.csv', 'Agent36_Convos.csv', 'Agent5_Convos.csv', 'Agent63_Convos.csv', 'Agent54_Convos.csv', 'Agent85_Convos.csv', 'Agent18_Convos.csv', 'Agent99_Convos.csv', 'Agent33_Convos.csv', 'Agent51_Convos.csv', 'Agent66_Convos.csv', 'Agent77_Convos.csv', 'Agent40_Convos.csv', 'Agent15_Convos.csv', 'Agent22_Convos.csv', 'Agent88_Convos.csv', 'Agent94_Convos.csv', 'Agent93_Convos.csv', 'Agent39_Convos.csv', 'Agent70_Convos.csv', 'Agent47_Convos.csv', 'Agent101_Convos.csv', 'Agent12_Convos.csv', 'Agent25_Convos.csv', 'Agent34_Convos.csv', 'Agent7_Convos.csv', 'Agent56_Convos.csv', 'Agent61_Convos.csv', 'Agent28_Convos.csv', 'Agent82_Convos.csv', 'Agent2_Convos.csv', 'Agent31_Convos.csv', 'Agent64_Convos.csv', 'Agent53_Convos.csv', 'Agent87_Convos.csv', 'Agent78_Convos.csv

TypeError: join() argument must be str, bytes, or os.PathLike object, not 'function'

In [ ]:
twitter.agentDict[1].conversationRecord.head()

In [ ]:
#Output conversation records per agent to separate CSV
convosGenDF = pd.DataFrame()
for agent in twitter.agentDict.values():
    convoRecDF = agent.conversationRecord   
    convoRecDF.to_csv('%s/Agent%s_Convos.csv' % (ind_results_path, agent.name), index=False)  

In [63]:
#Output conversation records of all agent to one CSV
genConvoRecDF = pd.DataFrame()
for agent in twitter.agentDict.values():
    genConvoRecDF = pd.concat([genConvoRecDF, agent.conversationRecord], ignore_index=True)
    
genConvoRecDF.to_csv('%s/Convos.csv' % (test_path), index=False)

In [64]:
#Output Agent DF
agentDF = pd.DataFrame(twitter.agentMatrix)
agentDF.to_csv('%s/Agents.csv' % (test_path), index=False)

In [65]:
agentDF.head(51)

,agent,agent_preference,num_connections,agent_connections,influence_status
1,1,i like the part when he got covid but still ha...,5,"[59, 40, 75, 63, 77]",False
2,2,this month i personally know people that have ...,19,"[85, 88, 98, 21, 64, 40, 78, 29, 47, 24, 33, 5...",False
3,3,don t go to a party with a house full of ppl n...,9,"[64, 67, 85, 45, 20, 59, 77, 30, 1]",False
4,4,continuing to mask up and distance reduces the...,27,"[61, 80, 17, 5, 20, 70, 46, 58, 9, 59, 33, 50,...",False
5,5,wholeheartedly agree in the last weeks i have ...,29,"[96, 12, 84, 59, 79, 48, 26, 61, 51, 20, 45, 4...",False
6,6,took my first flight in months for a work trip...,8,"[71, 26, 36, 38, 84, 3, 41, 81]",False
7,7,as someone who was there this weekend who live...,8,"[99, 33, 28, 95, 69, 93, 3, 49]",False
8,8,thanks ctl stay safe friends and wear mask if ...,20,"[30, 15, 44, 40, 32, 14, 64, 43, 82, 86, 16, 9...",False
9,9,mask protect other people he only cares about ...,22,"[50, 49, 67, 1, 4, 69, 40, 10, 24, 39, 71, 6, ...",False
10,10,please help by following the order so we can r...,23,"[57, 58, 54, 89, 81, 99, 43, 31, 85, 2, 53, 35...",False


In [104]:
#Convert Agent DF to Graph DF and Output Graph DF
agentDF_Graph_Nodes = agentDF[['agent']].copy()
agentDF_Graph_Nodes.rename(columns={'agent':'id'}, inplace=True)
agentDF_Graph_Nodes['label'] = agentDF_Graph_Nodes.loc[:, 'id']
#agentDF_Graph_Nodes.rename(columns={'agent':'id','agent_connections':'target'}, inplace=True)
agentDF_Graph = agentDF[['agent','agent_connections']].copy()
agentDF_Graph.rename(columns={'agent':'source','agent_connections':'target'}, inplace=True)
agentDF_Graph = agentDF_Graph.explode('target')
agentDF_Graph.to_csv('%s/Graph_Edges.csv' % (test_path), index=False)

In [105]:
#Output Sentiment DF
#sentimentDF.to_csv('%s/Sentiment.csv' % (test_path), index_label='timestep')
sentimentDF.head()

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.9623,0.3182,0.6070,0.5719,0.7351,0.4025,0.9253,0.8360,0.6808,0.9485,...,0.6223,0.8223,0.1027,0.4215,0.3182,0.7430,0.3612,0.5742,0.4576,-0.8081
1,0.9957,0.9968,0.9977,0.9909,0.9955,0.9985,0.9863,0.9849,0.9976,0.9985,...,0.9868,0.9979,0.8608,0.9843,0.3182,0.9933,0.9186,0.9860,0.9937,-0.9993
2,0.9988,0.9222,0.9946,0.9879,0.9853,0.9944,0.9976,0.9582,0.9969,0.9969,...,0.9946,0.9962,0.8979,0.9981,0.9998,0.8540,0.9911,0.9891,0.9987,-0.9993
3,0.9879,-0.8322,0.9799,0.9622,0.9898,0.6956,0.9971,0.9950,0.9022,0.9965,...,0.9919,-0.8168,0.9931,0.9983,0.9999,-0.9689,0.9902,0.9835,0.7802,-0.9993
4,0.9976,-0.9894,0.9926,0.9822,0.9244,0.0364,0.8182,0.9965,0.9482,0.9895,...,0.9858,0.9649,0.9980,0.9998,0.9999,-0.9877,0.9977,0.9692,-0.9922,-0.9993


In [106]:
sentiment_transposed = sentimentDF.transpose(copy=False)#.reset_index(drop=True)
sentiment_transposed.index = sentiment_transposed.index + 1
#sentiment_transposed = sentiment_transposed.T.reset_index(drop=True).T
#sentiment_transposed.columns = range(sentiment_transposed.columns.size)]
sentiment_transposed.tail()

,0,1,2,3,4,5,6,7,8,9,10,11
97,0.7430,0.9933,0.8540,-0.9689,-0.9877,0.9716,0.9768,0.9905,-0.8101,-0.9575,-0.9416,-0.6952
98,0.3612,0.9186,0.9911,0.9902,0.9977,0.9979,0.9988,0.9967,0.9981,0.9901,0.9988,0.9971
99,0.5742,0.9860,0.9891,0.9835,0.9692,0.9697,0.9857,0.9901,-0.9915,0.9661,0.6172,-0.4202
100,0.4576,0.9937,0.9987,0.7802,-0.9922,-0.9721,-0.9969,-0.9925,-0.9985,-0.9817,-0.9841,-0.9947
101,-0.8081,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993,-0.9993


In [107]:
#sentiment_transposed = sentimentDF.transpose(copy=False)
agentDF_Graph_Nodes = pd.concat([agentDF_Graph_Nodes,sentiment_transposed], axis=1).reindex(agentDF_Graph_Nodes.index)
agentDF_Graph_Nodes.to_csv('%s/Graph_Nodes.csv' % (test_path), index=False)

In [69]:
correspondenceTimeline = pd.DataFrame(twitter.correspondenceTimeline)
correspondenceTimeline.to_csv('%s/CorrespondenceTimeline_DF.csv' % (test_path), index=False)

In [547]:
correspondenceTimeline.head()

,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
0,"{'source': 3, 'target': 2, 'message': 'i wonde...","{'source': 3, 'target': 2, 'message': 'A lot o...","{'source': 3, 'target': 2, 'message': 'You hav...","{'source': 3, 'target': 2, 'message': 'In fact...","{'source': 3, 'target': 2, 'message': '> Is th...","{'source': 3, 'target': 2, 'message': 'It's al...","{'source': 3, 'target': 2, 'message': 'I'm so ...","{'source': 3, 'target': 2, 'message': ''Is the...","{'source': 3, 'target': 2, 'message': 'I'm so ...","{'source': 3, 'target': 2, 'message': '> We do...",...,"{'source': 3, 'target': 2, 'message': '""...I t...","{'source': 3, 'target': 2, 'message': '""I have...","{'source': 3, 'target': 2, 'message': 'The peo...","{'source': 3, 'target': 2, 'message': ''Making...","{'source': 3, 'target': 2, 'message': '[https:...","{'source': 3, 'target': 2, 'message': '""I'm so...","{'source': 3, 'target': 2, 'message': '[shorth...","{'source': 3, 'target': 2, 'message': '""The po...","{'source': 3, 'target': 2, 'message': '[NAME](...","{'source': 3, 'target': 2, 'message': '[NAME](..."
1,"{'source': 6, 'target': 1, 'message': 'this gu...","{'source': 6, 'target': 1, 'message': 'That's ...","{'source': 6, 'target': 1, 'message': 'I'm in ...","{'source': 6, 'target': 1, 'message': 'It's a ...","{'source': 6, 'target': 1, 'message': '>I don'...","{'source': 6, 'target': 1, 'message': 'I can t...","{'source': 6, 'target': 1, 'message': 'In this...","{'source': 6, 'target': 1, 'message': 'My fami...","{'source': 6, 'target': 1, 'message': 'It seem...","{'source': 6, 'target': 1, 'message': '> I fee...",...,"{'source': 6, 'target': 1, 'message': 'CANNOTA...","{'source': 6, 'target': 1, 'message': 'This is...","{'source': 6, 'target': 1, 'message': '""Hey si...","{'source': 6, 'target': 1, 'message': '[Neem] ...","{'source': 6, 'target': 1, 'message': '“If you...","{'source': 6, 'target': 1, 'message': 'The CDC...","{'source': 6, 'target': 1, 'message': 'A lot o...","{'source': 6, 'target': 1, 'message': 'Vaccine...","{'source': 6, 'target': 1, 'message': 'I do th...","{'source': 6, 'target': 1, 'message': 'I think..."
2,"{'source': 6, 'target': 4, 'message': 'i perso...","{'source': 6, 'target': 4, 'message': 'I was a...","{'source': 6, 'target': 4, 'message': 'A CP 'c...","{'source': 6, 'target': 4, 'message': 'I think...","{'source': 6, 'target': 4, 'message': '>I just...","{'source': 6, 'target': 4, 'message': '> What ...","{'source': 6, 'target': 4, 'message': 'Is ther...","{'source': 6, 'target': 4, 'message': 'I'm not...","{'source': 6, 'target': 4, 'message': 'What ab...","{'source': 6, 'target': 4, 'message': 'A nut j...",...,"{'source': 6, 'target': 4, 'message': 'Its a g...","{'source': 6, 'target': 4, 'message': 'So the ...","{'source': 6, 'target': 4, 'message': '>Why do...","{'source': 6, 'target': 4, 'message': '>The CD...","{'source': 6, 'target': 4, 'message': '[In all...","{'source': 6, 'target': 4, 'message': '>A CP'c...","{'source': 6, 'target': 4, 'message': 'There i...","{'source': 6, 'target': 4, 'message': '...and ...","{'source': 6, 'target': 4, 'message': 'That's ...","{'source': 6, 'target': 4, 'message': '>""Dad, ..."
3,"{'source': 7, 'target': 4, 'message': 'You are...","{'source': 7, 'target': 4, 'message': '[NAME] ...","{'source': 7, 'target': 4, 'message': 'Its lik...","{'source': 7, 'target': 4, 'message': 'Basical...","{'source': 7, 'target': 4, 'message': 'It’s no...","{'source': 7, 'target': 4, 'message': 'You kno...","{'source': 7, 'target': 4, 'message': 'Is this...","{'source': 7, 'target': 4, 'message': '""I thin...","{'source': 7, 'target': 4, 'message': '>(http:...","{'source': 7, 'target': 4, 'message': 'CANNOTA...",...,"{'source': 7, 'target': 4, 'message': 'Why the...","{'source': 7, 'target': 4, 'message': 'The fir...","{'source': 7, 'target': 4, 'message': '""You do...","{'source': 7, 'target': 4, 'message': '""Making...","{'source': 7, 'target': 4, 'message': 'CANNOTA...","{'source':

In [70]:
import json 
correspondenceTimeline_json = json.dumps(twitter.correspondenceTimeline, indent = 4) 
with open('%s/CorrespondenceTimeline_JSON.csv' % (test_path), "w") as outfile:
    outfile.write(correspondenceTimeline_json)
print(correspondenceTimeline_json)

{
    "1": [
        {
            "source": 3,
            "target": 1,
            "message": "What about going to a party with a house full of ppl not wasting mask during a pandemic?"
        },
        {
            "source": 6,
            "target": 3,
            "message": "Why does everyone do something that makes people feel safe? I am not a therapist, but I know people who do it. Its not a good idea to have a mask because you think people need a mask."
        },
        {
            "source": 7,
            "target": 3,
            "message": "I ran 5 miles to find this out a week ago."
        },
        {
            "source": 8,
            "target": 6,
            "message": "It seems like all the government is trying to stop the spread of the virus, and they are trying to stop it."
        },
        {
            "source": 8,
            "target": 7,
            "message": "Well that's an understatement. It's just a piece of crap."
        },
        {
            "so

In [10]:
import json 

# Opening JSON file
with open('/Users/zacpangan/Documents/Research/Code/Models/AgentNetworkSimulation/1 InfluencerSimulation - Jan2023-jvsc-9bd93d30-1f56-4cf1-acc3-ce44668ba50394cd1338-d3ce-49e4-885e-8aa63800e031/01-26-2023 22_27_22/100Agents10Timesteps_TestRun2_Jan26/CorrespondenceTimeline_JSON.json') as json_file:
    data = json.load(json_file)
#

In [12]:
correspondenceTimeline_preprocessed = pd.DataFrame(columns=['timestep', 'message'])
for k1, v1 in data.items():
    time = k1
    for dict_obj in v1:
        for k2, v2 in dict_obj.items():
                correspondenceTimeline_preprocessed = correspondenceTimeline_preprocessed.append({
                    'timestep': time, 'message': dict_obj.get("message")}, ignore_index=True)

correspondenceTimeline_preprocessed.head()
                

,timestep,message
0,1,What about going to a party with a house full ...
1,1,What about going to a party with a house full ...
2,1,What about going to a party with a house full ...
3,1,Why does everyone do something that makes peop...
4,1,Why does everyone do something that makes peop...


In [118]:

correspondenceTimeline_preprocessed['message'][22000]

">I wear a mask on a run. It's hard to tell when my face isn't shattered by a virus. I don't know why but this is why I can't get it. I've never been to the NSA or any other government agency, but it sounds like they're talking about a 'virus' that can spread over the nation. I think it's interesting that a lot of people are being manipulated to do this. It's just a joke,"

In [13]:
correspondenceTimeline_preprocessed.to_csv('%s/CorrespondenceTimeline_DF_preprocessed.csv' % (test_path), index=False)

In [72]:
twitter.agentDict[101].chats

{2: ["Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it.",
  "Wearing masks does not help in stopping the virus, it's useless and I hate it."],
 4: ["Wearing masks does not help in stopping the virus, it's useless an